In [1]:
import models.helpers as mh
import models.builders as mb
import inference.helpers as ih
import data.breathe_data as breathe_data
import data.smartcare_data as smartcare_data
import data.helpers as dh

import numpy as np
import pandas as pd

In [4]:
# df = breathe_data.load_meas_from_excel('BR_O2_FEV1')
df = breathe_data.build_O2_FEV1_FEF2575_PEF_df(2, remove_nan=True)
df.head()


*** Building O2Sat, FEV1, FEF2575 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  243
Number of rows:  48978
Number of FEV1 recordings: 41791
Number of FEF2575 recordings: 37068
Number of PEF recordings: 16755
Number of O2 Saturation recordings: 43930
Dropped 33610 entries with at least one NaN in subset ['O2 Saturation', 'FEV1', 'FEF2575', 'PEF']
This includes dropping 5048 entries with NaN O2 Saturation
This includes dropping 7187 entries with NaN FEV1
This includes dropping 11910 entries with

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2021-05-25,1.68,98.0,1.17,227.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,46.536607,100.874827
1,101,2021-05-26,1.65,98.0,1.06,236.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,45.705597,100.874827
2,101,2021-05-27,1.69,98.0,1.12,183.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,46.813611,100.874827
3,101,2021-05-28,1.67,98.0,1.08,175.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,46.259604,100.874827
4,101,2021-05-29,1.69,98.0,1.16,171.0,1.76,53,Male,173.0,3.610061,97.150104,48.752636,46.813611,100.874827


In [5]:
# Infer AR and IA for all data points given an individuals' age, sex, height, FEV1 and O2 saturation measurements
def infer_AR_IA_for_ID(df):
    df.reset_index(inplace=True)
    _, inf_alg, HFEV1, ecFEV1, AR, HO2Sat, _, IA, _, O2Sat = mb.o2sat_fev1_point_in_time_model_shared_healthy_vars(
        df.Height[0], df.Age[0], df.Sex[0]
    )

    def infer_and_unpack(ecfev1_obs, o2sat_obs):
        res = ih.infer_on_factor_graph(
            inf_alg,
            [AR, IA, HFEV1, HO2Sat],
            [[ecFEV1, ecfev1_obs], [O2Sat, o2sat_obs]],
        )
        return res[AR.name].values, res[IA.name].values, res[HFEV1.name].values, res[HO2Sat.name].values

    res = df.apply(
        lambda row: infer_and_unpack(row["ecFEV1"], row["O2 Saturation"]),
        axis=1,
    )
    return res


resraw = df.groupby("ID").apply(infer_AR_IA_for_ID)
# resraw = df.iloc[np.r_[10:13, 1000:1007]].groupby("ID").apply(infer_AR_IA_for_ID)
res = (
    resraw.apply(pd.Series)
    .reset_index()
    .rename(columns={0: "AR", 1: "IA", 2: "HFEV1", 3: "HO2Sat"})
    .drop(columns="level_1")
)
res

,ID,AR,IA,HFEV1,HO2Sat
0,101,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,101,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,101,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,101,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,101,"[0.00032111950996979114, 0.0004109729606994951...","[0.8452978344899926, 0.1425778679219038, 0.011...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
15363,357,"[0.018894458144689832, 0.023900270613168705, 0...","[0.5938219299610586, 0.32073461779482193, 0.07...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
15364,358,"[0.12393173080736078, 0.12818807612289895, 0.1...","[0.794036182578991, 0.18549455618183203, 0.019...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
15365,358,"[0.12393173080736081, 0.12818807612289898, 0.1...","[0.3561906667243055, 0.39015595419992494, 0.20...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.5970932163126048e-22, 4..."
15366,358,"[0.11364347821363134, 0.11951918572027004, 0.1...","[0.794036182578991, 0.18549455618183203, 0.019...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
AR = mh.VariableNode("Airway resistance (%)", 0, 90, 2, prior=None)
IA = mh.VariableNode("Inactive alveoli (%)", 0, 30, 1, prior=None)
HFEV1 = mh.VariableNode("Healthy FEV1 (L)", 1, 6, 0.05, prior=None)
HO2Sat = mh.VariableNode("Healthy O2 saturation (%)", 90, 100, 0.5, prior=None)


res["AR mean"] = res["AR"].apply(lambda x: AR.get_mean(x))
res["IA mean"] = res["IA"].apply(lambda x: IA.get_mean(x))
res["HFEV1 mean"] = res["HFEV1"].apply(lambda x: HFEV1.get_mean(x))
res["HO2Sat mean"] = res["HO2Sat"].apply(lambda x: HO2Sat.get_mean(x))

In [10]:
df.columns

Index(['ID', 'Date Recorded', 'FEV1', 'O2 Saturation', 'FEF2575', 'PEF',
       'ecFEV1', 'Age', 'Sex', 'Height', 'Predicted FEV1',
       'Healthy O2 Saturation', 'ecFEV1 % Predicted', 'FEV1 % Predicted',
       'O2 Saturation % Healthy'],
      dtype='object')

In [11]:
# Concatenate the 4 new columns to the original dataframe
df1 = pd.concat([df, res.drop(columns=["ID"])], axis=1)
df1.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,...,FEV1 % Predicted,O2 Saturation % Healthy,AR,IA,HFEV1,HO2Sat,AR mean,IA mean,HFEV1 mean,HO2Sat mean
0,101,2021-05-25,1.68,98.0,1.17,227.0,1.69,53,Male,173.0,...,46.536607,100.874827,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",47.601368,0.654616,3.285826,97.643061
1,101,2021-05-26,1.65,98.0,1.06,236.0,1.69,53,Male,173.0,...,45.705597,100.874827,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",47.601368,0.654616,3.285826,97.643061
2,101,2021-05-27,1.69,98.0,1.12,183.0,1.69,53,Male,173.0,...,46.813611,100.874827,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",47.601368,0.654616,3.285826,97.643061
3,101,2021-05-28,1.67,98.0,1.08,175.0,1.69,53,Male,173.0,...,46.259604,100.874827,"[0.00020140268126914501, 0.0002563950730509337...","[0.8561467340582659, 0.13340126187709733, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",47.601368,0.654616,3.285826,97.643061
4,101,2021-05-29,1.69,98.0,1.16,171.0,1.76,53,Male,173.0,...,46.813611,100.874827,"[0.00032111950996979114, 0.0004109729606994951...","[0.8452978344899926, 0.1425778679219038, 0.011...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",44.593062,0.667227,3.288958,97.621469


In [12]:
df1.columns

Index(['ID', 'Date Recorded', 'FEV1', 'O2 Saturation', 'FEF2575', 'PEF',
       'ecFEV1', 'Age', 'Sex', 'Height', 'Predicted FEV1',
       'Healthy O2 Saturation', 'ecFEV1 % Predicted', 'FEV1 % Predicted',
       'O2 Saturation % Healthy', 'AR', 'IA', 'HFEV1', 'HO2Sat', 'AR mean',
       'IA mean', 'HFEV1 mean', 'HO2Sat mean'],
      dtype='object')

In [6]:
# Save rawres as excel to ../../../../ExcelFiles
df1.to_excel(
    dh.get_path_to_main() + "ExcelFiles/BR/BR_O2_FEV1_FEF2575_PEF_inferred_AR_IA_HFEV1_HO2Sat.xlsx",
    index=False,
)

# Save arrays as objects
Arrays are currently saved as strings which is painful to process when reading from the file

In [ ]:
import pandas as pd
import numpy as np
import data.helpers as data_helpers

In [ ]:
data_helpers.load_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx",
    ["AR", "IA"],
).iloc[0].AR

In [ ]:
df = pd.read_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx"
)

In [ ]:
# Convert back to arrays
def str_to_array(s):
    s_cleaned = s.replace("\\n", "")
    return np.fromstring(s_cleaned[1:-1], sep=" ")


df["AR"] = df.AR.apply(str_to_array)
df["IA"] = df.IA.apply(str_to_array)

In [ ]:
df.iloc[0].AR

In [ ]:
df.to_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx",
    index=False,
)

In [ ]:
df = pd.read_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor.xlsx"
)

In [ ]:
df.AR.apply(json.loads)[0]